In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyl
import datetime
from openpyxl import load_workbook

df_june = pd.read_excel(r'D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\MIF Jun 2023.xlsx', 
                        sheet_name='MIF Jun 2023 running 28 Jun 23')

df_june_acct = pd.read_excel(r'D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\mif_mei_rev_v1.xlsx',
                        sheet_name='Acct Mapping')
#print(df_june.info())
print(df_june_acct.shape)

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow, Flow
from google.auth.transport.requests import Request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
credFile = r'D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\spreadsheet-stuff-387510-ad57904e68e8.json'

creds = Credentials.from_service_account_file(credFile, scopes=scopes)
gSpreadAuth = gspread.authorize(creds)
gAuth = GoogleAuth()
gDrive = GoogleDrive(gAuth)
gSpreadOpen = gSpreadAuth.open_by_key('1vPt9Lqkbj2FilGmVsDoPAv4BP-bs_7vkSO1mi-e0-V8')

gSpread_details_tab = gSpreadOpen.worksheet('Details')

In [ ]:
details_tab = gSpread_details_tab.get_all_records()

df_may = pd.DataFrame(details_tab)

print(df_may.info())

In [ ]:
# Acct Mapping
df_june_acct = df_june_acct.astype({'Acct':str})

map_sales = dict(df_june_acct.values)
df_june['Sales'] = df_june['Acct'].map(map_sales)
print(df_june.info())
print(map_sales)

In [ ]:
print(df_june.loc[df_june['Utilisasi '] == '-'].head(2))

In [ ]:
#df_gcs = df_june.loc[df_june['Utilisasi '] == '-']
#print(df_june['Class_Utilisasi'].unique())
#print(df_june.info())

#try:
#    if df_gcs['Utilisasi '].any() == '-':
#        df_gcs['Utilisasi '].replace('-', float(0), inplace=True)
#        df_gcs['Utilisasi '] = (df_gcs['Ave Tot  MoP BW'] + df_gcs['Ave Tot  MoP Color'])/(df_gcs['Speed'] * 60 * 8 * 23)
#except Exception as e:
#    print(e)
    
# THE MACHINE'S SPEED DATA IS NOT COMPLETED, CANNOT BE PROCEEDED

In [ ]:
df_june['InstDate'] = pd.to_datetime(df_june['InstDate'])

In [ ]:
#df_june['InstDate'] = pd.to_datetime(df_june['InstDate'], format='%Y-%m-%d')
df_june['InstDate'] = df_june['InstDate'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
df_june['InstDate'].head()
df_june.drop(columns='No', inplace=True)

df_june.to_excel(
    r'D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\MIF Juni Clean.xlsx',
    sheet_name='Details'
                )

In [ ]:
df_juni = pd.read_excel(
    r"D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\MIF Juni Clean.xlsx",
    sheet_name='Details (wo GCS)'
)
df_sales_data = df_juni[['Acct','Sales']]
print(len(list(df_sales_data['Acct'].unique())))

df_sales_data.drop_duplicates(subset='Acct', inplace=True)
print(df_sales_data.shape)

In [ ]:
df_sales_data.to_excel(r'D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\Sales Dataset\sales.xlsx')

In [ ]:
# Pivoting sales vs industry

df_juni['desc'] = df_juni['Industry Desc']
df_sales_ind = df_juni.pivot_table(index='Sales', columns='Industry Desc', values='Unit', aggfunc='sum').fillna(0).astype(int)
df_sales_ind

In [ ]:
# EDS devices only

df_juni['MBU'].unique()
df_juni_eds = df_juni.loc[(df_juni['MBU'] != 'GCS') & (df_juni['MBU'] != 'ES')]

df_juni.drop(columns='desc', inplace=True)
df_juni_eds.drop(columns='desc', inplace=True)

print(df_juni_eds.shape)

In [ ]:
# Identifying old devices

print(list(df_juni_eds['Umur Class'].unique()),'\n')
df_juni_eds_5 = df_juni_eds.loc[df_juni_eds['Umur Class'] == '5th - Up']
print(df_juni_eds_5['Family'].unique(),'\n')

# Identifying over utilized devices
df_juni_over = df_juni_eds.loc[df_juni_eds['Class_Utilisasi'] == '7% ~ Up']
print(df_juni_over.shape,'\n')
cek = df_juni_over.groupby('Sales')['Unit'].sum()
print(cek,'\n')

# Sales selling performance
sales_perf = df_juni_eds.pivot_table(index='Sales', columns='SaTy', values='Unit', aggfunc='sum')
print(sales_perf,'\n', sales_perf.values.sum(),'\n')

# Mono vs CLR
mono_clr = df_juni_eds.groupby('Mono/Color')['Unit'].sum()
print(mono_clr,'\n')

# Devices in EOS Zone
print(df_juni_eds['EOS'].unique(),'\n')
df_juni_eos = df_juni_eds.loc[(df_juni_eds['EOS'] == 'EOS') | (df_juni_eds['EOS'] == 'EOS Des 2024')]
print(df_juni_eos.shape)

In [ ]:
path = r'D:\\Data Shelter\Kerjaan\\Astragraphia\\BO1\\Revenue Analysis_jan-jul\\2023 Datasets\\MIF Juni Clean.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
df_juni_eds.to_excel(writer, sheet_name='Details (wo GCS)')
df_sales_ind.to_excel(writer, sheet_name='Pivot Ind (wo GCS)')
df_juni_eds_5.to_excel(writer, sheet_name='Oldies')
df_juni_over.to_excel(writer, sheet_name='Overwork')
sales_perf.to_excel(writer, sheet_name='Performance')
mono_clr.to_excel(writer, sheet_name='Mono vs CLR')
df_juni_eos.to_excel(writer, sheet_name='EOS')
writer.close()

In [ ]:
gSpreadJune = gSpreadAuth.open_by_key('10m9Yl4R1jpbc72JaK8X7bQ5yfsPIZ0VD5nf9vzsc518')
gSpread_details_rev = gSpreadJune.worksheet('Details')
gSpreadJune = gSpread_details_rev.get_all_records()
df_june_new = pd.DataFrame(gSpreadJune)
df_june_new.info()

In [ ]:
#Pivot
df_june_new['desc'] = df_june_new['Industry Desc']
df_ind_juni = df_june_new.pivot_table(index='Sales', columns='Industry Desc', values='Unit', aggfunc='sum').fillna(0).astype(int)
df_june_new.drop(columns='desc', inplace=True)
print(df_ind_juni,'\n')

# Identifying old devices
print(list(df_june_new['Umur Class'].unique()),'\n')
df_juni_5 = df_june_new.loc[df_june_new['Umur Class'] == '5th - Up']
print(df_juni_5['Family'].unique(),'\n')

# Identifying over utilized devices
df_over = df_june_new.loc[df_june_new['Class_Utilisasi'] == '7% ~ Up']
print(df_over.shape,'\n')
cek = df_over.groupby('Sales')['Unit'].sum()
print(cek,'\n')

# Sales selling performance
df_perf = df_june_new.pivot_table(index='Sales', columns='SaTy', values='Unit', aggfunc='sum')
print(df_perf,'\n', df_perf.values.sum(),'\n')

# Mono vs CLR
monoVsClr = df_june_new.groupby('Mono/Color')['Unit'].sum()
print(monoVsClr,'\n')

# Devices in EOS Zone
print(df_june_new['EOS'].unique(),'\n')
juni_eos = df_june_new.loc[(df_june_new['EOS'] == 'EOS') | (df_june_new['EOS'] == 'EOS Des 2024')]
print(juni_eos.shape)

In [ ]:
path = r'D:\\Data Shelter\Kerjaan\\Astragraphia\\BO1\\Revenue Analysis_jan-jul\\2023 Datasets\\MIF Juni Clean - Copy.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
df_june_new.to_excel(writer, sheet_name='Details (wo GCS)')
df_ind_juni.to_excel(writer, sheet_name='Pivot Ind (wo GCS)')
df_juni_5.to_excel(writer, sheet_name='Oldies')
df_over.to_excel(writer, sheet_name='Overwork')
df_perf.to_excel(writer, sheet_name='Performance')
monoVsClr.to_excel(writer, sheet_name='Mono vs CLR')
juni_eos.to_excel(writer, sheet_name='EOS')
writer.close()

In [ ]:
# New Mapping

print(len(df_june_new['Acct'].unique()))
df_june_ns = df_june_new[['Acct','Sales']]
df_june_ns.drop_duplicates(subset='Acct', inplace=True)
print(df_june_ns.shape)

df_juni_rd = pd.read_excel(
    r"D:\Data Shelter\Kerjaan\Astragraphia\BO1\Revenue Analysis_jan-jul\2023 Datasets\MIF Juni Clean.xlsx",
    sheet_name='Details'
)

In [ ]:
rd_sales_map = df_juni[['Acct','Sales']]
rd_sales_map = rd_sales_map.astype({'Acct':str})
rd_sales_dict = dict(rd_sales_map.values)
df_juni_rd['Sales'] = df_juni_rd['Acct'].map(rd_sales_dict)
print(rd_sales_map.info())
print(df_juni_rd.info())

In [ ]:
path = r'D:\\Data Shelter\Kerjaan\\Astragraphia\\BO1\\Revenue Analysis_jan-jul\\2023 Datasets\\MIF Juni Clean.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
df_juni_rd.to_excel(writer, sheet_name='Details')
writer.close()

In [ ]:
df_june_v = pd.read_excel(
    r'D:\\Data Shelter\Kerjaan\\Astragraphia\\BO1\\Revenue Analysis_jan-jul\\2023 Datasets\\MIF Juni Clean.xlsx',
    sheet_name = 'Details'
)
df_june_sales_v = df_june_v[['Acct','Sales']].astype(str)
df_june_sales_v.drop_duplicates(subset='Acct', inplace=True)

path = r'D:\\Data Shelter\Kerjaan\\Astragraphia\\BO1\\Revenue Analysis_jan-jul\\2023 Datasets\\MIF Juni Clean.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
df_june_sales_v.to_excel(writer, sheet_name='Acct mapping')
writer.close()